### Load Packages

In [2]:
import pandas as pd
import numpy as np
import os
import warnings
import re
import datetime 

warnings.simplefilter("ignore")

pd.options.mode.chained_assignment = None

### Import Excel Files

In [5]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\Semi-Monthly Closed Invoice\2022 Semi-Monthly Closed Invoice\03_2022'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

### ACCESSORIAL Generate Semi-Monthly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [3]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail = df_detail[['CustomerID', 'DocNo', 'WISE Facility', 'Header Billing Period Start', 'Bill Pay Category']]
        
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] == 'ACCESSORIAL']
        
    
    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
    
    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 
                                                            'Billing Period Start', 'DocNo'])
    
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)

(287, 7)
(206, 7)


In [4]:
df_list[len(df_list) - 1].sort_values('Accounting_CustomerID')

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Billing Period Start,DocNo
6991,AIRINC0001,AIRINC0001,Grand Prairie,ACCESSORIAL,"AIRHOUSE, INC.",03/20/2022,19054142
6930,AIRINC0001,AIRINC0001,Valley View,ACCESSORIAL,"AIRHOUSE, INC.",03/20/2022,19054228
1450,AMEFUT0001,IBUPOW0001,Valley View,ACCESSORIAL,I BUY POWER,03/16/2022,19054921
100,AMILYN0001,AMILYN0001,Valley View,ACCESSORIAL,"AMIEE LYNN, LNC.",03/16/2022,19055039
4254,ANDLIF0001,ANDLIF0002,Willow,ACCESSORIAL,ANDEAN LIFE,03/16/2022,19054638
...,...,...,...,...,...,...,...
6407,VIZIO0001,VIZIO0001,Indiana,ACCESSORIAL,VIZIO,03/16/2022,19054520
7103,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",03/22/2022,19053844
6990,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",03/20/2022,19054141
5135,ZYLACO0001,VIZIO0001,Indiana,ACCESSORIAL,Zylux Acoustic Corp.,03/16/2022,19054487


In [5]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.drop_duplicates()

df_output.columns = df_output.columns.str.replace('_x', '')
df_output.columns = df_output.columns.str.replace('_y', '')

(1709, 9)


In [6]:
df_output

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Billing Period Start,DocNo,Billing Period Start,DocNo
0,GENHOL0001,GENHOL0001,Seabrook,ACCESSORIAL,Generac Holdings Inc.,03/01/2022,19053994.0,03/16/2022,19055340.0
1,NOBHOU0001,NOBHOU0001,Alessandro,ACCESSORIAL,Noble House Furniture,03/01/2022,19053825.0,NaN,NaN
2,RADFLY0001,RADFLY0001,Innovation,ACCESSORIAL,RADIO FLYER,03/01/2022,19053764.0,03/16/2022,19054700.0
3,JBHHOM0001,JBHHOM0001,Lakewood,ACCESSORIAL,JB HUNT - HOMEWERKS,03/14/2022,19053610.0,03/28/2022,19054362.0
4,JBHHOM0001,JBHHOM0001,Lakewood,ACCESSORIAL,JB HUNT - HOMEWERKS,03/14/2022,19053610.0,03/25/2022,19054227.0
...,...,...,...,...,...,...,...,...,...
1704,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,NaN,03/23/2022,19053915.0
1705,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,NaN,03/21/2022,19053831.0
1706,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",NaN,NaN,03/20/2022,19054141.0
1707,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",NaN,NaN,03/22/2022,19053844.0


### Generate Semi-Monthly Report with Billing Count and without Invoice Number

In [7]:
df_drop_iv = df_output.copy(deep = True)
df_drop_iv.drop('DocNo', inplace = True, axis = 1)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).nunique(axis = 1)
df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Billing Period Start,Billing Period Start,Count
0,GENHOL0001,GENHOL0001,Seabrook,ACCESSORIAL,Generac Holdings Inc.,03/01/2022,03/16/2022,2
1,NOBHOU0001,NOBHOU0001,Alessandro,ACCESSORIAL,Noble House Furniture,03/01/2022,NaN,1
2,RADFLY0001,RADFLY0001,Innovation,ACCESSORIAL,RADIO FLYER,03/01/2022,03/16/2022,2
3,JBHHOM0001,JBHHOM0001,Lakewood,ACCESSORIAL,JB HUNT - HOMEWERKS,03/14/2022,03/28/2022,2
4,JBHHOM0001,JBHHOM0001,Lakewood,ACCESSORIAL,JB HUNT - HOMEWERKS,03/14/2022,03/25/2022,2
...,...,...,...,...,...,...,...,...
1703,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,03/23/2022,1
1705,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,03/21/2022,1
1706,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",NaN,03/20/2022,1
1707,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",NaN,03/22/2022,1


In [8]:
df_count.sort_values(['Count'])

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Billing Period Start,Billing Period Start,Count
1708,USBINT0001,USBINT0001,New Jersey,ACCESSORIAL,USB INTERNATIONAL LTD,NaN,03/16/2022,1
1538,SUNGLO0001,SUNGLO0001,Houston,ACCESSORIAL,"SUNRISE GLOBAL MARKETING, LLC",03/01/2022,NaN,1
1498,OUTER0001,OUTER0001,Kansas,ACCESSORIAL,"OUTER, INC.",03/01/2022,NaN,1
1500,EDLFIN0001,EDLFIN0001,Charleston,ACCESSORIAL,EDLOE FINCH LLC,03/01/2022,NaN,1
1576,SGCOMP0001,SGCOMP0001,Innovation,ACCESSORIAL,THE SG COMPANIES,03/06/2022,NaN,1
...,...,...,...,...,...,...,...,...
715,CHRCVS0001,CHRCVS0001,Willow,ACCESSORIAL,C.H.Robinson-CVS,03/08/2022,03/24/2022,2
714,CHRCVS0001,CHRCVS0001,Willow,ACCESSORIAL,C.H.Robinson-CVS,03/08/2022,03/25/2022,2
712,CHRCVS0001,CHRCVS0001,Willow,ACCESSORIAL,C.H.Robinson-CVS,03/08/2022,03/28/2022,2
621,CHRCVS0001,CHRCVS0001,Willow,ACCESSORIAL,C.H.Robinson-CVS,03/10/2022,03/16/2022,2


In [9]:
# month = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

month = np.array(['Mar'])
semi_month = np.array([])
for m in month:
    semi_month = np.append(semi_month, m + '_Mid')
    semi_month = np.append(semi_month, m + '_End')

bp = np.where(df_count.columns.values == 'Billing Period Start')

df_count.columns.values[bp] = semi_month

df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_End,Count
0,GENHOL0001,GENHOL0001,Seabrook,ACCESSORIAL,Generac Holdings Inc.,03/01/2022,03/16/2022,2
1,NOBHOU0001,NOBHOU0001,Alessandro,ACCESSORIAL,Noble House Furniture,03/01/2022,NaN,1
2,RADFLY0001,RADFLY0001,Innovation,ACCESSORIAL,RADIO FLYER,03/01/2022,03/16/2022,2
3,JBHHOM0001,JBHHOM0001,Lakewood,ACCESSORIAL,JB HUNT - HOMEWERKS,03/14/2022,03/28/2022,2
4,JBHHOM0001,JBHHOM0001,Lakewood,ACCESSORIAL,JB HUNT - HOMEWERKS,03/14/2022,03/25/2022,2
...,...,...,...,...,...,...,...,...
1703,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,03/23/2022,1
1705,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,03/21/2022,1
1706,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",NaN,03/20/2022,1
1707,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",NaN,03/22/2022,1


In [10]:
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()
df_count = df_count.drop_duplicates(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_End,Count
1508,MIDAME0001,MIDAME0001,Alessandro,ACCESSORIAL,MIDEA AMERICA CORP,03/01/2022,NaN,1
1,NOBHOU0001,NOBHOU0001,Alessandro,ACCESSORIAL,NOBLE HOUSE FURNITURE,03/01/2022,NaN,1
1700,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,03/31/2022,1
1500,EDLFIN0001,EDLFIN0001,Charleston,ACCESSORIAL,EDLOE FINCH LLC,03/01/2022,NaN,1
1679,OUTER0001,OUTER0001,Charleston,ACCESSORIAL,"OUTER, INC.",03/10/2022,NaN,1
...,...,...,...,...,...,...,...,...
1517,CONLUM0001,CONLUM0001,Willow,ACCESSORIAL,CONCANNON LUMBER COMPANY,03/01/2022,03/16/2022,2
1696,DDCNUT0001,DDCNUT0001,Willow,ACCESSORIAL,NUTRIBINS LLC,NaN,03/16/2022,1
1514,GILING0001,GILING0002,Willow,ACCESSORIAL,GILLCO INGREDIENTS - CA,03/01/2022,03/16/2022,2
1518,GKSKA0001,GKSKAGG001,Willow,ACCESSORIAL,GK SKAGGS,03/01/2022,03/16/2022,2


In [11]:
df_count.to_excel("03_2022 Semi-Monthly Report Accessorial Count.xlsx", index = False, header = True)

### Export Semi-Monthly Report As Excel File

In [12]:
# month = np.array(['Jan', 'Feb'])

# semi_month = np.array([])
# for m in month:
#     semi_month = np.append(semi_month, m + '_Mid')
#     semi_month = np.append(semi_month, m + '_End')

bp1 = np.where(df_output.columns.values == 'Billing Period Start')
dn1 = np.where(df_output.columns.values == 'DocNo')

dn_month = [x + '_DocNo' for x in semi_month]

df_output.columns.values[bp1] = semi_month
df_output.columns.values[dn1] = dn_month

In [13]:
df_output

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_Mid_DocNo,Mar_End,Mar_End_DocNo
0,GENHOL0001,GENHOL0001,Seabrook,ACCESSORIAL,Generac Holdings Inc.,03/01/2022,19053994.0,03/16/2022,19055340.0
1,NOBHOU0001,NOBHOU0001,Alessandro,ACCESSORIAL,Noble House Furniture,03/01/2022,19053825.0,NaN,NaN
2,RADFLY0001,RADFLY0001,Innovation,ACCESSORIAL,RADIO FLYER,03/01/2022,19053764.0,03/16/2022,19054700.0
3,JBHHOM0001,JBHHOM0001,Lakewood,ACCESSORIAL,JB HUNT - HOMEWERKS,03/14/2022,19053610.0,03/28/2022,19054362.0
4,JBHHOM0001,JBHHOM0001,Lakewood,ACCESSORIAL,JB HUNT - HOMEWERKS,03/14/2022,19053610.0,03/25/2022,19054227.0
...,...,...,...,...,...,...,...,...,...
1704,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,NaN,03/23/2022,19053915.0
1705,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,NaN,03/21/2022,19053831.0
1706,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",NaN,NaN,03/20/2022,19054141.0
1707,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",NaN,NaN,03/22/2022,19053844.0


In [14]:
df_output.groupby(['Accounting_CustomerID']).count().sort_values(['CustomerID'], ascending=False).head(12)

,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_Mid_DocNo,Mar_End,Mar_End_DocNo
Accounting_CustomerID,,,,,,,,
CHRCVS0001,1088,1088,1088,1088,1088,1088,1036,1036
JBHHOM0001,231,231,231,231,231,231,231,231
NORSTA0001,98,98,98,98,98,98,98,98
CHRPHI0001,34,34,34,34,34,34,34,34
BANINT0001,33,33,33,33,33,33,33,33
INNCOR0001,25,25,25,25,25,25,25,25
PEPSIC0001,24,24,24,24,24,24,24,24
SDSGLO0001,13,13,13,13,2,2,11,11
LAJOLG0001,13,13,13,13,12,12,13,13


In [15]:
df_output = df_output.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_output['WISE Facility'] = df_output['WISE Facility'].str.title()
df_output['Bill To'] = df_output['Bill To'].str.upper()
df_output

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_Mid_DocNo,Mar_End,Mar_End_DocNo
1508,MIDAME0001,MIDAME0001,Alessandro,ACCESSORIAL,MIDEA AMERICA CORP,03/01/2022,19053339.0,NaN,NaN
1,NOBHOU0001,NOBHOU0001,Alessandro,ACCESSORIAL,NOBLE HOUSE FURNITURE,03/01/2022,19053825.0,NaN,NaN
1700,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,NaN,03/31/2022,19054421.0
1701,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,NaN,03/31/2022,19054420.0
1702,SDSGLO0001,SDSGLO0001,Bolingbrook,ACCESSORIAL,"SAMSUNG SDS AMERICA, INC.",NaN,NaN,03/31/2022,19054419.0
...,...,...,...,...,...,...,...,...,...
1696,DDCNUT0001,DDCNUT0001,Willow,ACCESSORIAL,NUTRIBINS LLC,NaN,NaN,03/16/2022,19054792.0
1514,GILING0001,GILING0002,Willow,ACCESSORIAL,GILLCO INGREDIENTS - CA,03/01/2022,19053210.0,03/16/2022,19055312.0
1518,GKSKA0001,GKSKAGG001,Willow,ACCESSORIAL,GK SKAGGS,03/01/2022,19053218.0,03/16/2022,19054617.0
1470,OUTER0001,OUTER0001,Willow,ACCESSORIAL,"OUTER, INC.",03/01/2022,19053345.0,03/16/2022,19054951.0


In [16]:
df_output.to_excel("03_2022 Semi-Monthly Report Accessorial.xlsx", index = False, header = True)

In [17]:
# writer = pd.ExcelWriter('2022 Semi-Monthly Report Accessorial.xlsx', engine = 'xlsxwriter')

In [18]:
# df_count.to_excel(writer, index = False, header = True, sheet_name = 'Summary')

In [19]:
# df_output.to_excel(writer, index = False, header = True, sheet_name = 'Detail 01-02')

In [20]:
# writer.save()

# MOVE FILES TO OUTPUT!

### HANDLING Generate Semi-Monthly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [21]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail = df_detail[['CustomerID', 'DocNo', 'WISE Facility', 'Header Billing Period Start', 'Bill Pay Category']]
    
    
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] == 'HANDLING']
    
    
    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
    


    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 
                                                            'Billing Period Start', 'DocNo'])
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)

(928, 7)
(558, 7)


In [22]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.drop_duplicates()

df_output.columns = df_output.columns.str.replace('_x', '')
df_output.columns = df_output.columns.str.replace('_y', '')

(58740, 9)


### Generate Semi-Monthly Report with Billing Count and without Invoice Number

In [23]:
df_drop_iv = df_output.copy(deep = True)
df_drop_iv.drop('DocNo', inplace = True, axis = 1)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).nunique(axis = 1)
df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Billing Period Start,Billing Period Start,Count
0,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/07/2022,NaN,1
1,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/03/2022,NaN,1
2,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/14/2022,NaN,1
3,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/15/2022,NaN,1
8,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/11/2022,NaN,1
...,...,...,...,...,...,...,...,...
58735,TORRAS0001,CONDAT0001,New Jersey,HANDLING,LECTA - TORRASPAPEL,NaN,03/16/2022,1
58736,HIHLOG0001,HIHLOG0001,New Jersey,HANDLING,"HIH LOGISTICS, INC",NaN,03/16/2022,1
58737,MINUSA0001,MINUSA0001,New Jersey,HANDLING,Miniso Depot CA,NaN,03/20/2022,1
58738,LEPCHK0001,LEPCHK0001,QUA,HANDLING,LENOVO (UNITED STATES) INC. -US01,NaN,03/23/2022,1


In [24]:
month = np.array(['Mar'])
semi_month = np.array([])
for m in month:
    semi_month = np.append(semi_month, m + '_Mid')
    semi_month = np.append(semi_month, m + '_End')

bp = np.where(df_count.columns.values == 'Billing Period Start')

df_count.columns.values[bp] = semi_month

df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_End,Count
0,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/07/2022,NaN,1
1,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/03/2022,NaN,1
2,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/14/2022,NaN,1
3,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/15/2022,NaN,1
8,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/11/2022,NaN,1
...,...,...,...,...,...,...,...,...
58735,TORRAS0001,CONDAT0001,New Jersey,HANDLING,LECTA - TORRASPAPEL,NaN,03/16/2022,1
58736,HIHLOG0001,HIHLOG0001,New Jersey,HANDLING,"HIH LOGISTICS, INC",NaN,03/16/2022,1
58737,MINUSA0001,MINUSA0001,New Jersey,HANDLING,Miniso Depot CA,NaN,03/20/2022,1
58738,LEPCHK0001,LEPCHK0001,QUA,HANDLING,LENOVO (UNITED STATES) INC. -US01,NaN,03/23/2022,1


In [25]:
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()
df_count = df_count.drop_duplicates(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_End,Count
58507,MIDAME0001,MIDAME0001,Alessandro,HANDLING,MIDEA AMERICA CORP,03/01/2022,03/26/2022,2
5245,NOBHOU0001,NOBHOU0001,Alessandro,HANDLING,NOBLE HOUSE FURNITURE,03/01/2022,NaN,1
10902,SDSGLO0001,SDSGLO0001,Alessandro,HANDLING,"SAMSUNG SDS AMERICA, INC.",03/01/2022,03/16/2022,2
58470,HOMWOR0001,HOMWOR0001,Bolingbrook,HANDLING,"HOMEWERKS WORLDWIDE, LLC",03/01/2022,NaN,1
58710,LIPCOM0001,LIPCOM0001,Bolingbrook,HANDLING,"LIPPERT COMPONENTS, INC.",NaN,03/16/2022,1
...,...,...,...,...,...,...,...,...
58727,DDCNUT0001,DDCNUT0001,Willow,HANDLING,NUTRIBINS LLC,NaN,03/16/2022,1
58514,GILING0001,GILING0002,Willow,HANDLING,GILLCO INGREDIENTS - CA,03/01/2022,03/16/2022,2
58518,GKSKA0001,GKSKAGG001,Willow,HANDLING,GK SKAGGS,03/01/2022,03/16/2022,2
3844,LIPCOM0001,LIPCOM0001,Willow,HANDLING,"LIPPERT COMPONENTS, INC.",03/01/2022,03/16/2022,2


In [26]:
df_count.to_excel("03_2022 Semi-Monthly Report Handling Count.xlsx", index = False, header = True)

### Export Semi-Monthly Report As Excel File

In [27]:
# month = np.array(['Jan'])

# semi_month = np.array([])
# for m in month:
#     semi_month = np.append(semi_month, m + '_Mid')
#     semi_month = np.append(semi_month, m + '_End')

bp1 = np.where(df_output.columns.values == 'Billing Period Start')
dn1 = np.where(df_output.columns.values == 'DocNo')

dn_month = [x + '_DocNo' for x in semi_month]

df_output.columns.values[bp1] = semi_month
df_output.columns.values[dn1] = dn_month

In [28]:
df_output.groupby(['Accounting_CustomerID']).count().sort_values(['CustomerID'], ascending=False)
df_output

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_Mid_DocNo,Mar_End,Mar_End_DocNo
0,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/07/2022,19055438.0,NaN,NaN
1,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/03/2022,19055437.0,NaN,NaN
2,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/14/2022,19055282.0,NaN,NaN
3,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/15/2022,19055279.0,NaN,NaN
4,CHRCVS0001,CHRCVS0001,Savannah,HANDLING,C.H.Robinson-CVS,03/15/2022,19055277.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
58735,TORRAS0001,CONDAT0001,New Jersey,HANDLING,LECTA - TORRASPAPEL,NaN,NaN,03/16/2022,19054496.0
58736,HIHLOG0001,HIHLOG0001,New Jersey,HANDLING,"HIH LOGISTICS, INC",NaN,NaN,03/16/2022,19054463.0
58737,MINUSA0001,MINUSA0001,New Jersey,HANDLING,Miniso Depot CA,NaN,NaN,03/20/2022,19054136.0
58738,LEPCHK0001,LEPCHK0001,QUA,HANDLING,LENOVO (UNITED STATES) INC. -US01,NaN,NaN,03/23/2022,19053921.0


In [29]:
df_output = df_output.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_output['WISE Facility'] = df_output['WISE Facility'].str.title()
df_output['Bill To'] = df_output['Bill To'].str.upper()
df_output

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_Mid_DocNo,Mar_End,Mar_End_DocNo
58507,MIDAME0001,MIDAME0001,Alessandro,HANDLING,MIDEA AMERICA CORP,03/01/2022,19053339.0,03/26/2022,19055434.0
58508,MIDAME0001,MIDAME0001,Alessandro,HANDLING,MIDEA AMERICA CORP,03/01/2022,19053339.0,03/19/2022,19055432.0
5245,NOBHOU0001,NOBHOU0001,Alessandro,HANDLING,NOBLE HOUSE FURNITURE,03/01/2022,19053825.0,NaN,NaN
10902,SDSGLO0001,SDSGLO0001,Alessandro,HANDLING,"SAMSUNG SDS AMERICA, INC.",03/01/2022,19053461.0,03/16/2022,19055308.0
10903,SDSGLO0001,SDSGLO0001,Alessandro,HANDLING,"SAMSUNG SDS AMERICA, INC.",03/01/2022,19053461.0,03/16/2022,19055011.0
...,...,...,...,...,...,...,...,...,...
5234,LIPCOM0001,LIPCOM0001,Willow,HANDLING,"LIPPERT COMPONENTS, INC.",03/01/2022,19053765.0,03/16/2022,19055344.0
5235,LIPCOM0001,LIPCOM0001,Willow,HANDLING,"LIPPERT COMPONENTS, INC.",03/01/2022,19053765.0,03/16/2022,19055343.0
5236,LIPCOM0001,LIPCOM0001,Willow,HANDLING,"LIPPERT COMPONENTS, INC.",03/01/2022,19053765.0,03/16/2022,19055342.0
5237,LIPCOM0001,LIPCOM0001,Willow,HANDLING,"LIPPERT COMPONENTS, INC.",03/01/2022,19053765.0,03/16/2022,19054868.0


In [30]:
df_output.to_excel("03_2022 Semi-Monthly Report Handling.xlsx", index = False, header = True)

In [31]:
# writer1 = pd.ExcelWriter('2022 Semi-Monthly Report Handling.xlsx', engine = 'xlsxwriter')
# df_count.to_excel(writer1, index = False, header = True, sheet_name = 'Summary')
# df_output.to_excel(writer1, index = False, header = True, sheet_name = 'Detail 01-02')
# writer1.save()

# MOVE FILES TO OUTPUT!

### Others Generate Semi-Monthly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [6]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail = df_detail[['CustomerID', 'DocNo', 'WISE Facility', 'Header Billing Period Start', 'Bill Pay Category']]
        
        
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] != 'STORAGE INCOME']
    
    df_select_detail['Bill Pay Category'] = np.where((df_select_detail['Bill Pay Category'] != 'HANDLING') & 
                                                     (df_select_detail['Bill Pay Category'] != 'ACCESSORIAL'), 
                                           'OTHERS', df_select_detail['Bill Pay Category'])
    
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] == 'OTHERS']
        
 

    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
    
 

    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 
                                                            'Billing Period Start', 'DocNo'])
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)

(457, 7)
(231, 7)


In [7]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.drop_duplicates()

df_output.columns = df_output.columns.str.replace('_x', '')
df_output.columns = df_output.columns.str.replace('_y', '')

(52146, 9)


### Generate Semi-Monthly Report with Billing Count and without Invoice Number

In [8]:
df_drop_iv = df_output.copy(deep = True)
df_drop_iv.drop('DocNo', inplace = True, axis = 1)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).nunique(axis = 1)
df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Billing Period Start,Billing Period Start,Count
0,TRESMA0001,TPVTRA0001,Via Baron,OTHERS,TPV TRANSLOAD,03/01/2022,03/16/2022,2
1,OUTER0001,OUTER0001,Willow,OTHERS,"OUTER, INC.",03/01/2022,03/16/2022,2
2,VIZIO0001,VIZIO0001,Indiana,OTHERS,VIZIO,03/06/2022,03/20/2022,2
3,VIZIO0001,VIZIO0001,Indiana,OTHERS,VIZIO,03/09/2022,03/20/2022,2
4,VIBESB0001,VIBESB0001,Valley View,OTHERS,VIBESBASE,03/06/2022,NaN,1
...,...,...,...,...,...,...,...,...
52141,INTKNO0001,INTKNO0001,Fontana,OTHERS,INTRADECO-KNOTHE,NaN,03/24/2022,1
52142,ZOOLLC0001,ZOOLLC0001,Murphy,OTHERS,"ZOOMGET, LLC",NaN,03/21/2022,1
52143,MIDAME0001,MIDAME0001,Alessandro,OTHERS,Midea America Corp,NaN,03/21/2022,1
52144,MIDAME0001,MIDAME0001,Alessandro,OTHERS,Midea America Corp,NaN,03/17/2022,1


In [9]:
month = np.array(['Mar'])
semi_month = np.array([])
for m in month:
    semi_month = np.append(semi_month, m + '_Mid')
    semi_month = np.append(semi_month, m + '_End')

bp = np.where(df_count.columns.values == 'Billing Period Start')

df_count.columns.values[bp] = semi_month

df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_End,Count
0,TRESMA0001,TPVTRA0001,Via Baron,OTHERS,TPV TRANSLOAD,03/01/2022,03/16/2022,2
1,OUTER0001,OUTER0001,Willow,OTHERS,"OUTER, INC.",03/01/2022,03/16/2022,2
2,VIZIO0001,VIZIO0001,Indiana,OTHERS,VIZIO,03/06/2022,03/20/2022,2
3,VIZIO0001,VIZIO0001,Indiana,OTHERS,VIZIO,03/09/2022,03/20/2022,2
4,VIBESB0001,VIBESB0001,Valley View,OTHERS,VIBESBASE,03/06/2022,NaN,1
...,...,...,...,...,...,...,...,...
52141,INTKNO0001,INTKNO0001,Fontana,OTHERS,INTRADECO-KNOTHE,NaN,03/24/2022,1
52142,ZOOLLC0001,ZOOLLC0001,Murphy,OTHERS,"ZOOMGET, LLC",NaN,03/21/2022,1
52143,MIDAME0001,MIDAME0001,Alessandro,OTHERS,Midea America Corp,NaN,03/21/2022,1
52144,MIDAME0001,MIDAME0001,Alessandro,OTHERS,Midea America Corp,NaN,03/17/2022,1


In [10]:
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()
df_count = df_count.drop_duplicates(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_End,Count
52143,MIDAME0001,MIDAME0001,Alessandro,OTHERS,MIDEA AMERICA CORP,NaN,03/21/2022,1
52105,SDSGLO0001,SDSGLO0001,Bolingbrook,OTHERS,"SAMSUNG SDS AMERICA, INC.",03/11/2022,NaN,1
52091,FLAANT0001,FLAANT0001,Fontana,OTHERS,FLAG & ANTHEM,03/06/2022,03/20/2022,2
52141,INTKNO0001,INTKNO0001,Fontana,OTHERS,INTRADECO-KNOTHE,NaN,03/24/2022,1
52109,SIMMOD0001,SIMMOD0001,Fontana,OTHERS,SIMPLE MODERN,03/10/2022,03/31/2022,2
52103,SDSGLO0001,SDSGLO0001,Hazelton,OTHERS,"SAMSUNG SDS AMERICA, INC.",03/11/2022,NaN,1
52090,BRAWES0001,BRAWES0001,Houston,OTHERS,"BRAZORIA WEST SOLAR PROJECT, LLC",03/15/2022,NaN,1
52116,ROYING0001,ROYING0001,Houston,OTHERS,ROYAL INGREDIENTS GROUP USA INC.,03/09/2022,NaN,1
52138,TCLTEC0003,VIZIO0001,Indiana,OTHERS,"TCL - TONLY ACOUSTIC, INC",NaN,03/24/2022,1
2,VIZIO0001,VIZIO0001,Indiana,OTHERS,VIZIO,03/06/2022,03/20/2022,2


In [11]:
df_count.to_excel("03_2022 Semi-Monthly Report Others Count.xlsx", index = False, header = True)

### Export Semi-Monthly Report As Excel File

In [12]:
# month = np.array(['Jan'])

# semi_month = np.array([])
# for m in month:
#     semi_month = np.append(semi_month, m + '_Mid')
#     semi_month = np.append(semi_month, m + '_End')

bp1 = np.where(df_output.columns.values == 'Billing Period Start')
dn1 = np.where(df_output.columns.values == 'DocNo')

dn_month = [x + '_DocNo' for x in semi_month]

df_output.columns.values[bp1] = semi_month
df_output.columns.values[dn1] = dn_month

In [13]:
df_output.groupby(['Accounting_CustomerID']).count().sort_values(['CustomerID'], ascending=False)
df_output

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_Mid_DocNo,Mar_End,Mar_End_DocNo
0,TRESMA0001,TPVTRA0001,Via Baron,OTHERS,TPV TRANSLOAD,03/01/2022,19053538.0,03/16/2022,19055543.0
1,OUTER0001,OUTER0001,Willow,OTHERS,"OUTER, INC.",03/01/2022,19053524.0,03/16/2022,19055328.0
2,VIZIO0001,VIZIO0001,Indiana,OTHERS,VIZIO,03/06/2022,19053522.0,03/20/2022,19054422.0
3,VIZIO0001,VIZIO0001,Indiana,OTHERS,VIZIO,03/09/2022,19052724.0,03/20/2022,19054422.0
4,VIBESB0001,VIBESB0001,Valley View,OTHERS,VIBESBASE,03/06/2022,19053521.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
52141,INTKNO0001,INTKNO0001,Fontana,OTHERS,INTRADECO-KNOTHE,NaN,NaN,03/24/2022,19053995.0
52142,ZOOLLC0001,ZOOLLC0001,Murphy,OTHERS,"ZOOMGET, LLC",NaN,NaN,03/21/2022,19053841.0
52143,MIDAME0001,MIDAME0001,Alessandro,OTHERS,Midea America Corp,NaN,NaN,03/21/2022,19053840.0
52144,MIDAME0001,MIDAME0001,Alessandro,OTHERS,Midea America Corp,NaN,NaN,03/17/2022,19053536.0


In [14]:
df_output = df_output.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_output['WISE Facility'] = df_output['WISE Facility'].str.title()
df_output['Bill To'] = df_output['Bill To'].str.upper()
df_output

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Mar_Mid,Mar_Mid_DocNo,Mar_End,Mar_End_DocNo
52143,MIDAME0001,MIDAME0001,Alessandro,OTHERS,MIDEA AMERICA CORP,NaN,NaN,03/21/2022,19053840.0
52144,MIDAME0001,MIDAME0001,Alessandro,OTHERS,MIDEA AMERICA CORP,NaN,NaN,03/17/2022,19053536.0
52105,SDSGLO0001,SDSGLO0001,Bolingbrook,OTHERS,"SAMSUNG SDS AMERICA, INC.",03/11/2022,19052933.0,NaN,NaN
52106,SDSGLO0001,SDSGLO0001,Bolingbrook,OTHERS,"SAMSUNG SDS AMERICA, INC.",03/09/2022,19052742.0,NaN,NaN
52107,SDSGLO0001,SDSGLO0001,Bolingbrook,OTHERS,"SAMSUNG SDS AMERICA, INC.",03/09/2022,19052741.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4592,ASCRET0001,ASCRET0001,Via Baron,OTHERS,ASCENA RETAIL GROUP,03/01/2022,19052298.0,03/16/2022,19053591.0
52113,BEXENT0001,BEXENT0001,Via Baron,OTHERS,"BEXCO ENTERPRISES, INC",03/10/2022,19052841.0,NaN,NaN
52114,BEXENT0001,BEXENT0001,Via Baron,OTHERS,"BEXCO ENTERPRISES, INC",03/08/2022,19052640.0,NaN,NaN
0,TRESMA0001,TPVTRA0001,Via Baron,OTHERS,TPV TRANSLOAD,03/01/2022,19053538.0,03/16/2022,19055543.0


In [15]:
df_output.to_excel("03_2022 Semi-Monthly Report Others.xlsx", index = False, header = True)

In [ ]:
# bp = np.where(df_count.columns.values == 'Billing Period Start')
# df_count.columns.values[bp] = semi_month

In [ ]:
# writer2 = pd.ExcelWriter('2022 Semi-Monthly Report Others.xlsx', engine = 'xlsxwriter')
# df_count.to_excel(writer2, index = False, header = True, sheet_name = 'Summary')
# df_output.to_excel(writer2, index = False, header = True, sheet_name = 'Detail 01-02')
# writer2.save()

# MOVE FILES TO OUTPUT!

### Get Accessorial Items

In [16]:
df_list = []

df_combine = pd.DataFrame()

month = ['Mar', 'Mar']

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail = df_detail[['CustomerID', 'DocNo', 'WISE Facility', 'Header Billing Period Start', 'Bill Pay Category',
                                 'BillingCode', 'Qty', 'Cost']]
    
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] == 'ACCESSORIAL']
    
    
    df_select_detail['BillingCode'] = df_select_detail['BillingCode'].str.lower()
    
    df_select_detail['Item'] = np.nan
    
     
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains(r'pallet[^4]|pallet\s[^4]', 
                                                                      regex=True, na=False), 'Item'] = 'Customized Size Pallet'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('chep', na=False), 'Item'] = 'CHEP Pallet'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('peco', na=False), 'Item'] = 'PECO Pallet'   
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('48x40', na=False), 'Item'] = 'Grade B Pallet'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('air bag', na=False), 'Item'] = 'Air Bag'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('brace', na=False), 'Item'] = 'Block And Brace'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('bol', na=False), 'Item'] = 'BOL'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('cancel', na=False), 'Item'] = 'Cancellation'
        
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('load bar', na=False), 'Item'] = 'Load Bar'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('appointment', na=False), 'Item'] = 'Missed Appointment'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('placard', na=False), 'Item'] = 'Placard'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('strap', na=False), 'Item'] = 'Plastic Strap'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('rush', na=False), 'Item'] = 'Rush Order'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('serial number', na=False), 'Item'] = 'Serial Number Tracking' 
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('sheet', na=False), 'Item'] = 'Slip Sheet'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('wrap', na=False), 'Item'] = 'Stretch Wrap'

    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('box', na=False), 'Item'] = 'Brown Box'   
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('label', na=False), 'Item'] = 'Label'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('labor', na=False), 'Item'] = 'Labor'  
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('palletizing and', na=False), 
                         'Item'] = 'Palletizing And Wrapping'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains(r'(?=.*document)', 
                                                                     regex=True, na=False), 'Item'] = 'Picture/Photo/Document'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains(r'(?=.*photo)', 
                                                                     regex=True, na=False), 'Item'] = 'Picture/Photo/Document'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains(r'(?=.*picture)', 
                                                                     regex=True, na=False), 'Item'] = 'Picture/Photo/Document'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('grade a', na=False), 'Item'] = 'Grade A Pallet'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('grade b', na=False), 'Item'] = 'Grade B Pallet'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('corner', na=False), 'Item'] = 'Corner Board'
        
            
    df_select_detail['Item'] = df_select_detail['Item'].fillna('Others')
    
    
    df_sum_detail = df_select_detail.groupby(['CustomerID', 'DocNo', 'WISE Facility', 
                                                'Header Billing Period Start', 'Bill Pay Category', 
                                                'BillingCode', 'Item'])['Qty', 'Cost'].sum().reset_index()
    
        
    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
    

    df_combine_header_detail = pd.merge(df_sum_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 'Billing Period Start', 
                                                            'DocNo', 'BillingCode', 'Item', 'Qty', 'Cost'])
    
    df_combine_month['Billing Period Start'] = month[excel]    
    
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)
    
    
    df_combine = df_combine.append(df_combine_month)
    
    print(df_combine.shape)

(798, 11)
(798, 11)
(681, 11)
(1479, 11)


In [17]:
df_list[len(df_list) - 1].sort_values('CustomerID')

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Billing Period Start,DocNo,BillingCode,Item,Qty,Cost
0,AIRINC0001,AIRINC0001,Grand Prairie,ACCESSORIAL,"AIRHOUSE, INC.",Mar,19054142,grade b pallet,Grade B Pallet,7.0,103.25
1,AIRINC0001,AIRINC0001,Grand Prairie,ACCESSORIAL,"AIRHOUSE, INC.",Mar,19054142,"literature (brochures, leaflets, manuals, gift...",Others,509.0,50.90
2,AIRINC0001,AIRINC0001,Valley View,ACCESSORIAL,"AIRHOUSE, INC.",Mar,19054228,"literature (brochures, leaflets, manuals, gift...",Others,4.0,0.40
3,AMILYN0001,AMILYN0001,Valley View,ACCESSORIAL,"AMIEE LYNN, LNC.",Mar,19055039,"accessorial noaction per unit, materialtype,sh...",Label,19.0,4.37
4,AMILYN0001,AMILYN0001,Valley View,ACCESSORIAL,"AMIEE LYNN, LNC.",Mar,19055039,"accessorial noaction per unit, materialtype,uc...",Label,13854.0,3186.42
...,...,...,...,...,...,...,...,...,...,...,...
648,VIZIO0001,VIZIO0001,Indiana,ACCESSORIAL,VIZIO,Mar,19054488,palletblock,Customized Size Pallet,124.0,1010.60
649,CTTUSA0001,VIZIO0001,Indiana,ACCESSORIAL,COMPETITION TEAM TECHNOLOGY USA INC.,Mar,19054516,"accessorial noaction per unit, materialtype,co...",Corner Board,261.0,1657.35
638,TCLTEC0003,VIZIO0001,Indiana,ACCESSORIAL,"TCL - Tonly Acoustic, Inc",Mar,19054486,pallet48x40,Grade B Pallet,22.0,300.08
679,ZOOLLC0001,ZOOLLC0001,Murphy,ACCESSORIAL,"ZOOMGET, LLC",Mar,19053844,supplies,Others,2025.0,2430.00


In [18]:
df_combine = df_combine.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_combine['WISE Facility'] = df_combine['WISE Facility'].str.title()
df_combine['Bill To'] = df_combine['Bill To'].str.upper()
df_combine

,Accounting_CustomerID,CustomerID,WISE Facility,Bill Pay Category,Bill To,Billing Period Start,DocNo,BillingCode,Item,Qty,Cost
604,MIDAME0001,MIDAME0001,Alessandro,ACCESSORIAL,MIDEA AMERICA CORP,Mar,19053339,"accessorial cancel order per order, ispicked,yes;",Cancellation,8.0,80.00
605,MIDAME0001,MIDAME0001,Alessandro,ACCESSORIAL,MIDEA AMERICA CORP,Mar,19053339,grade a pallet,Grade A Pallet,1.0,23.00
606,MIDAME0001,MIDAME0001,Alessandro,ACCESSORIAL,MIDEA AMERICA CORP,Mar,19053339,grade b pallet,Grade B Pallet,965.0,14233.75
607,MIDAME0001,MIDAME0001,Alessandro,ACCESSORIAL,MIDEA AMERICA CORP,Mar,19053339,shipping label,Label,1790.0,537.00
608,MIDAME0001,MIDAME0001,Alessandro,ACCESSORIAL,MIDEA AMERICA CORP,Mar,19053339,stretch wrap,Stretch Wrap,3342.0,13368.00
...,...,...,...,...,...,...,...,...,...,...,...
649,OUTER0001,OUTER0001,Willow,ACCESSORIAL,"OUTER, INC.",Mar,19053345,plastic strapping,Plastic Strap,356.0,1246.00
650,OUTER0001,OUTER0001,Willow,ACCESSORIAL,"OUTER, INC.",Mar,19053345,stretch wrap,Stretch Wrap,356.0,1424.00
383,OUTER0001,OUTER0001,Willow,ACCESSORIAL,"OUTER, INC.",Mar,19054951,labels,Label,390.0,117.00
384,OUTER0001,OUTER0001,Willow,ACCESSORIAL,"OUTER, INC.",Mar,19054951,plastic strapping,Plastic Strap,390.0,1365.00


In [19]:
df_combine.to_excel("03_2022 Accessorial Items Cost.xlsx", index = False, header = True)

# MOVE FILES TO OUTPUT!